In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter

import os

In [2]:
from pathlib import Path

# Configurable parts
SYMBOL = "USDJPY"
SAMPLE_TYPE = "time"
MINUTES = 1
DOLLAR_THRESHOLD = "115009542m"

START_DATE = "20210101"
END_DATE = "20241231"

# Build base name
if SAMPLE_TYPE == "dollar":
    RESAMPLED_NAME = f"{SYMBOL}-{DOLLAR_THRESHOLD}-dollar-{START_DATE}-{END_DATE}"
else:
    RESAMPLED_NAME = f"{SYMBOL}-{MINUTES}m-{START_DATE}-{END_DATE}"

# Base directories
BASE_DIR = Path("../data")
RESAMPLED_DIR = BASE_DIR / "resampled"
PROCESSED_DIR = BASE_DIR / "processed"
EVENTS_DIR = BASE_DIR / "events"

# Final paths
RESAMPLED_FILE_PATH = RESAMPLED_DIR / f"{RESAMPLED_NAME}.pkl"
PROCESSED_FILE_PATH = PROCESSED_DIR / f"{RESAMPLED_NAME}-processed.pkl"

In [3]:
%%time
df = pd.read_pickle(PROCESSED_FILE_PATH)

CPU times: user 755 μs, sys: 77.5 ms, total: 78.3 ms
Wall time: 80.7 ms


In [4]:
df.shape

(1487157, 70)

In [5]:
df.head()

,open,high,low,close,volume,spread,close_delta,close_return,close_log_return,ret_mean_5,...,dom,month,hour_sin,hour_cos,dow_sin,dow_cos,dom_sin,dom_cos,month_sin,month_cos
timestamp,,,,,,,,,,,,,,,,,,,,,
2021-01-04 00:34:00,103.1065,103.1065,103.0950,103.0995,1.330400e+11,0.002651,-0.0070,-0.000068,-0.000068,-0.000026,...,4,1,0.0,1.0,0.0,1.0,0.724793,0.688967,0.5,0.866025
2021-01-04 00:35:00,103.0990,103.0990,103.0695,103.0805,2.210200e+11,0.002410,-0.0190,-0.000184,-0.000184,-0.000038,...,4,1,0.0,1.0,0.0,1.0,0.724793,0.688967,0.5,0.866025
2021-01-04 00:36:00,103.0795,103.0940,103.0785,103.0880,1.112500e+11,0.002410,0.0075,0.000073,0.000073,-0.000035,...,4,1,0.0,1.0,0.0,1.0,0.724793,0.688967,0.5,0.866025
2021-01-04 00:37:00,103.0875,103.0880,103.0680,103.0745,1.644000e+11,0.002150,-0.0135,-0.000131,-0.000131,-0.000055,...,4,1,0.0,1.0,0.0,1.0,0.724793,0.688967,0.5,0.866025
2021-01-04 00:38:00,103.0760,103.0835,103.0705,103.0705,2.340300e+11,0.002316,-0.0040,-0.000039,-0.000039,-0.000070,...,4,1,0.0,1.0,0.0,1.0,0.724793,0.688967,0.5,0.866025


In [6]:
df.columns

Index(['open', 'high', 'low', 'close', 'volume', 'spread', 'close_delta',
       'close_return', 'close_log_return', 'ret_mean_5', 'ret_mean_10',
       'ret_mean_15', 'ret_mean_20', 'log_volume', 'ema5', 'ema5_slope',
       'ema20', 'ema20_slope', 'ema50', 'ema50_slope', 'ema100',
       'ema100_slope', 'atr14', 'atr25', 'atr50', 'log_atr14', 'atr14_percent',
       'atr14_adjusted_return', 'log_atr25', 'atr25_percent',
       'atr25_adjusted_return', 'log_atr50', 'atr50_percent',
       'atr50_adjusted_return', 'adx14', 'plus_di14', 'minus_di14', 'bb_upper',
       'bb_lower', 'bb_mavg', 'bb_width', 'bb_position', 'donchian_upper',
       'donchian_lower', 'donchian_mid', 'donchian_width', 'stoch_k',
       'stoch_d', 'rsi14', 'rsi14_slope', 'rsi25', 'rsi25_slope', 'rsi50',
       'rsi50_slope', 'macd', 'macd_signal', 'macd_diff', 'unix_time', 'hour',
       'dow', 'dom', 'month', 'hour_sin', 'hour_cos', 'dow_sin', 'dow_cos',
       'dom_sin', 'dom_cos', 'month_sin', 'month_cos'],
 

## Calculate Daily Volatility

In [7]:
vol = df['close_log_return'].rolling(window=60).std() # 計算 60 分鐘的 標準差
vol.mean()

np.float64(0.0001261218359079752)

## Get Events Start Time (t0)

In [8]:
import numpy as np
import pandas as pd
from tqdm import tqdm

def get_tevents_optimized(data: pd.Series, threshold: float) -> pd.DatetimeIndex:
    # get tEvents using CUSUM filter
    values = data.values
    timestamps = data.index

    s_pos = np.zeros_like(values)
    s_neg = np.zeros_like(values)

    t_events_mask = np.zeros_like(values, dtype=bool)

    cum_pos, cum_neg = 0.0, 0.0

    for i in tqdm(range(len(values))):
        cum_pos = max(0.0, cum_pos + values[i])
        cum_neg = min(0.0, cum_neg + values[i])
        s_pos[i] = cum_pos
        s_neg[i] = cum_neg

        if cum_pos > threshold:
            t_events_mask[i] = True
            cum_pos = 0.0
        if cum_neg < -threshold:
            t_events_mask[i] = True
            cum_neg = 0.0

    return timestamps[t_events_mask]


In [9]:
EVENT_NAME = 'CUSUM'
THRESHOLD = vol.mean()*2
EVENT_PATH = EVENTS_DIR / f"{RESAMPLED_NAME}_{EVENT_NAME}_{THRESHOLD:.2e}.pkl"

In [10]:
%%time

t_events = get_tevents_optimized(df['close_log_return'].iloc[1:], threshold=THRESHOLD)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1487156/1487156 [00:00<00:00, 3599240.38it/s]

CPU times: user 414 ms, sys: 11.8 ms, total: 426 ms
Wall time: 426 ms


In [11]:
t_events

DatetimeIndex(['2021-01-04 00:38:00', '2021-01-04 00:40:00',
               '2021-01-04 00:43:00', '2021-01-04 00:48:00',
               '2021-01-04 00:50:00', '2021-01-04 00:52:00',
               '2021-01-04 01:06:00', '2021-01-04 01:16:00',
               '2021-01-04 01:21:00', '2021-01-04 01:35:00',
               ...
               '2024-12-30 22:29:00', '2024-12-30 23:00:00',
               '2024-12-30 23:06:00', '2024-12-30 23:10:00',
               '2024-12-30 23:12:00', '2024-12-30 23:17:00',
               '2024-12-30 23:18:00', '2024-12-30 23:35:00',
               '2024-12-30 23:40:00', '2024-12-30 23:47:00'],
              dtype='datetime64[ns]', name='timestamp', length=248981, freq=None)

In [12]:
EVENT_PATH

PosixPath('../data/events/USDJPY-1m-20210101-20241231_CUSUM_2.52e-04.pkl')

In [13]:
t_events.to_series().to_pickle(EVENT_PATH)